In [ ]:
%matplotlib notebook

import os
import numpy as np
import matplotlib.pyplot as plt
import mne

创建结果存放目录

In [ ]:
def mkdir(path):
 
    path=path.strip()# 去除首位空格
    path=path.rstrip("\\")# 去除尾部 \ 符号
  
    isExists=os.path.exists(path)# 判断路径是否存在
  
    if not isExists:
        os.makedirs(path)
        print(path+' 创建成功')
        return True
    else:
        print(path+' 目录已存在')
        return False
  
# 定义要创建的目录
mkpath="results"
# 调用函数
mkdir(mkpath)

# Preprocessing

1. Loading data

In [ ]:
data_folder = 'raw_data'
dataFile = 'n170_data.set'
data_file = os.path.join(data_folder,dataFile)
data = mne.io.read_raw_eeglab(data_file, eog = 'auto')

In [ ]:
data.drop_channels('EOG')

In [ ]:
# check data manually
fig = data.plot(duration=5, n_channels=10, scalings=dict(eeg=80e-6))
plt.show()

2.Channel location

In [ ]:
montage = mne.channels.read_custom_montage('standard-10-5-cap385.elp')
data.set_montage(montage)

3. Resampling

In [ ]:
data_downsampled = data.resample(sfreq=250)

'''
This will reduce the timing precision of events

To avoid this reduction in precision, the suggested pipeline for processing final data to be analyzed is:

low-pass the data with mne.io.Raw.filter.
Extract epochs with mne.Epochs.
Decimate the Epochs object using mne.Epochs.decimate or the decim argument to the mne.Epochs object.

We also provide the convenience methods mne.Epochs.resample and mne.Evoked.resample to downsample or upsample data, but these are less optimal because they will introduce edge artifacts into every epoch, whereas filtering the raw data will only introduce edge artifacts only at the start and end of the recording.
'''

4. Filtering

In [ ]:
data_bandpass = data_downsampled.filter(l_freq=1, h_freq=35)
# data_notch = data_downsampled.notch_filter(freqs=50)
# data_downsampled.filter(None, 50., fir_design='firwin')

In [ ]:
# check data manually
# fig = data_bandpass.plot(duration=5, n_channels=64, scalings=dict(eeg=80e-6))
# plt.show()

5. Reference

In [ ]:
data_mastoid_ref = data_bandpass.set_eeg_reference(ref_channels=['M1', 'M2'], ch_type='eeg')

In [ ]:
# check data manually
# fig = data_mastoid_ref.plot(duration=5, n_channels=30, scalings=dict(eeg=80e-6))
# plt.show()

In [ ]:
# Rejecting EOG & ECG with SSP
from mne.preprocessing import (compute_proj_ecg, compute_proj_eog)

# Compute SSP/PCA projections for EOG artifacts
eog_projs, _ = compute_proj_eog(data_mastoid_ref, n_eeg=1, reject=None,
                                no_proj=True)
data_rejEOG = data_mastoid_ref.add_proj(eog_projs)

In [ ]:
ecg_projs, _ = compute_proj_ecg(data_rejEOG, n_eeg=1, reject=None, ch_name=, 
                                no_proj=True)
data_rejECG = data_rejEOG.add_proj(ecg_projs)

6. Epoch

In [ ]:
events, event_id = mne.events_from_annotations(data)
events_pick = mne.pick_events(events, exclude=[1,6,7,8])
# Mapping Event IDs to trial descriptors
event_dict = {'face/up': 2, 'chair/up': 3, 'face/down': 4,'chair/down': 5}

In [ ]:
# Creating Epoched data
epochs = mne.Epochs(data_mastoid_ref, events_pick, event_id=event_dict, 
                    tmin=-0.2, tmax=0.8, preload=True)#baseline correction was automatically applied 

In [ ]:
# # Reject bad epochs during configuration
# reject = dict(eeg=100e-6)
# epochs = mne.Epochs(data_mastoid_ref, events_pick, event_id=event_dict, reject=reject, 
#                     tmin=-0.2, tmax=0.8, preload=True)#baseline correction was automatically applied 

In [ ]:
# # You can also reject after constructing epochs
# reject.update({'eeg': 80e-6})
# epochs.drop_bad(reject=reject)

In [ ]:
# Check & reject epoch manually
# fig = epochs.plot(picks=['eeg'], scalings=dict(eeg=100e-6), n_epochs=2, n_channels=63, block=True)
# plt.show()

In [ ]:
# # 使用 autoreject 库
# # Autoreject (global) can compute the rejection dictionary automatically
from autoreject import get_rejection_threshold  # noqa
reject = get_rejection_threshold(epochs, ch_types='eeg')
print(reject)

In [ ]:
epochs_tmp1=epochs.copy()
epochs_tmp1.drop_bad(reject=reject)

In [ ]:
# 使用 autoreject 库
# Autoreject (local) finds per channel thresholds:
from autoreject import AutoReject

n_interpolates = np.array([1, 2, 4])
consensus = np.linspace(0.5, 1.0, 6)

ar = AutoReject(n_interpolates, consensus, thresh_method='random_search',
                random_state=42)

epochs_tmp2=epochs.copy()
# ar.fit(epochs['auditory/left'])
ar.fit(epochs_tmp2)

#look at the rejection thresholds for each channel
for ch_name in epochs.info['ch_names'][:5]:
     print('%s: %s' % (ch_name, ar.threshes_[ch_name]))

7.ICA

In [ ]:
from mne.preprocessing import (ICA, corrmap)

In [ ]:
ica = ICA(n_components=61, max_pca_components=61,random_state=None, method='infomax')
ica.fit(epochs)

# 或
# ica = mne.preprocessing.ICA（）

In [ ]:
# epochs.load_data()
ica.plot_sources(epochs, stop=3)
plt.show()

In [ ]:
ica.plot_components(inst=epochs)
plt.show()

In [ ]:
# Selecting ICA components manually
ica.exclude = [2]  # indices chosen based on various plots above

# ica.apply() changes the Raw object in-place, so let's make a copy first:
epoch_data = epochs.copy()
ica.apply(epoch_data)

In [ ]:
# ica.plot_properties(epochs, picks=ica.exclude)

8. Reject bad epochs

In [ ]:
fig = epoch_data.plot(picks=['eeg'], scalings=dict(eeg=100e-6), n_epochs=2, n_channels=63, block=True)
plt.show()

# ERP analysis

In [ ]:
'''
EvokedArray(data, info, tmin=0.0, comment='', nave=1, kind='average', verbose=None)[source]
data: array of shape (n_channels, n_times)
'''

In [ ]:
# 保证两个条件使用等量的trial数（有问题，待调整）
epoch_data.equalize_event_counts(['face','chair'])

face_evoked = epoch_data['face'].average()#MNE-Python can select based on partial matching of /-separated epoch labels
chair_evoked = epoch_data['chair'].average()

In [ ]:
# 保存 ERP 数据
face_evoked.save('results/face_eeg-ave.fif')  # save evoked data to disk
chair_evoked.save('results/chair_eeg-ave.fif')

差异波

In [ ]:
diff_evoked = mne.combine_evoked([face_evoked, chair_evoked], weights=[1,-1])
# diff_evoked.plot(picks=['P8'])
# plt.show()

# OR
# diff_data = face_evoked.data-chair_evoked.data

Butterfly plot

In [ ]:
# Butterfly plot
fig = plt.figure()
ax2 = fig.add_subplot(121)
ax3 = fig.add_subplot(122)
face_evoked.plot(gfp=True, spatial_colors=True, axes=ax2) #default exclude='bads'
chair_evoked.plot(gfp=True, spatial_colors=True, axes=ax3)
plt.show()

In [ ]:
# Joint plot
face_evoked.plot_joint(times=0.17)
plt.show()

Scalp maps

In [ ]:
# scalp topographies
# single time point
fig=face_evoked.plot_topomap(ch_type='eeg', times=0.17, average=0.01, colorbar=True)#outlines='skirt'to see the topography stretched beyond the head circle
fig.text(0.5, 0.05, 'average from 165-175 ms', ha='center')
plt.show()

In [ ]:
# scalp topographies
# multiple time points
times = np.arange(0.1, 0.3, 0.02)#间隔0.02s
face_evoked.plot_topomap(ch_type='eeg', times=times, colorbar=True,
                         ncols=5, nrows='auto')
plt.show()

In [ ]:
# Animating the topomap
times = np.arange(0.1, 0.5, 0.01)
fig, anim = face_evoked.animate_topomap(
    times=times, ch_type='eeg', frame_rate=2, time_unit='s', blit=False, 
    butterfly=True)

ERP traces

In [ ]:
# Comparing Evoked objects
mne.viz.plot_compare_evokeds([face_evoked, chair_evoked], picks='P8', ylim=dict(eeg=[-20,20]))
plt.show()

In [ ]:
# Topographical subplots
mne.viz.plot_compare_evokeds([face_evoked, chair_evoked], picks='eeg', axes='topo')
# OR
# mne.viz.plot_evoked_topo([face_evoked, chair_evoked])

Evoked arithmetic (e.g. differences)

In [ ]:
# create and plot difference ERP
joint_kwargs = dict(ts_args=dict(time_unit='s'),
                    topomap_args=dict(time_unit='s'))
mne.combine_evoked([face_evoked, chair_evoked], weights=[1, -1]).plot_joint(**joint_kwargs)
plt.show()

Peak detection

In [ ]:
# View evoked response
# times = 1e3 * epochs.times  # time in miliseconds

ch_max_name, latency, amplitude = face_evoked.get_peak(ch_type='eeg', tmin=0.15, tmax=0.2, mode='neg', return_amplitude=True)

face_evoked.plot(picks=ch_max_name)
plt.show()